In [6]:
# !pip install pandas
# !pip install requests
# !pip install geopython

In [1]:
import pandas as pd
import re

arquivo = 'EndereçoEscolasRecife.csv'

df = pd.read_csv(arquivo)

# Função para remover os últimos 8 dígitos
def remover_ultimos_digitos(endereco):
    return endereco[:-8] if len(endereco) >= 8 else endereco

# Função para extrair os dígitos usando regex
def extrair_cep(endereco):
    padrao_cep = re.compile(r'\b\d{8}\b')
    match = padrao_cep.search(endereco)
    if match:
        return match.group()
    else:
        return None

# Aplicar a função para criar a nova coluna "CEP"
df['CEP'] = df['Endereço'].apply(extrair_cep)

# Atualizar os valores na coluna 'Endereço'
df['Endereço'] = df['Endereço'].apply(remover_ultimos_digitos)

# Verificar se tem valores duplicados
df.duplicated().any()


False

In [16]:
# adress = 'Escola ' + df['Escola'].iloc[0] + ', ' + \
#     df['Endereço'].iloc[0] + ', ' + df['Bairro'].iloc[0] + \
#     ', ' + df['CEP'].iloc[0] + ' Recife-PE'
# adress

'Escola ALMIRANTE SOARES DUTRA, CAMUTANGA 180 , CABANGA, 50090380 Recife-PE'

In [ ]:
import requests

# Função para obter coordenadas usando a API Nominatim
def obter_coordenadas_nominatim(row):
    usuario_nominatim = 'User'
    endpoint = 'https://nominatim.openstreetmap.org/search'

    # Construir o endereço completo
    endereco_completo = f'Escola {row["Escola"]}, {row["Endereço"]}, {row["Bairro"]}, {row["CEP"]} Recife-PE'

    parametros = {
        'format': 'json',
        'limit': 1,
        'q': endereco_completo,
        'email': usuario_nominatim
    }

    resposta = requests.get(endpoint, params=parametros)
    dados = resposta.json()

    if dados and 'lat' in dados[0] and 'lon' in dados[0]:
        latitude = dados[0]['lat']
        longitude = dados[0]['lon']
        return latitude, longitude
    else:
        return None, None
    
# Criar duas novas colunas para latitude e longitude
df['Latitude'], df['Longitude'] = zip(*df.apply(obter_coordenadas_nominatim, axis=1))

df.to_csv('LatLongEscolasNominatim.csv', index=False)

In [8]:
# coordenadas = pd.read_csv('LatLongEscolas.csv')

# coordenadas.isnull().sum()

# Definindo os valores
parte = 427-384
total = 427

porcentagem = (parte / total) * 100

print(f"Somente {porcentagem:.2f}% das coordendas preenchidas usando Nominatim")

Somente 10.07% das coordendas preenchidas usando Nominatim


In [3]:
df

,idEscola,Escola,Endereço,Bairro,Telefone,RPA,CEP
0,1,ALMIRANTE SOARES DUTRA,CAMUTANGA 180,CABANGA,8133553880,1,50090380
1,2,ANA ROSA FALCAO DE CARVALHO,JOAO LYRA S/N,SANTO AMARO,8133556546,1,50050550
2,3,BOLSA ESCOLA MUNICIPAL,GENERAL ABREU E LIMA S/N,SANTO AMARO,NaN,1,50040210
3,4,CAP - BLOCO A DEGEP,FREI MATIAS TEVIS S/N,ILHA DO LEITE,NaN,1,50050470
4,5,CAP - BLOCO A DEGRE,FREI MATIAS TEVIS S/N,ILHA DO LEITE,NaN,1,50050470
...,...,...,...,...,...,...,...
422,423,UTEC MOVEL 6,ALMIRANTE NELSON FERNANDES S/N,BOA VIAGEM,8133550416,6,51030230
423,424,VILA DOS MILAGRES,SAO MIGUEL 14 MILAGRES,COHAB,8133554866,6,51345600
424,425,VILA OPERARIA DO RECIFE,EMILIO MONTEIRO FONSECA 1530,IBURA,8133556579,6,51240490
425,426,VILA SESAMO,VALE DO CARIRI 65 UR 05,COHAB,8133553160,6,51320180


In [10]:
import requests
import pandas as pd
# Função para obter coordenadas usando a API OpenCage Geocoding
def obter_coordenadas_opencage(row):
    api_key_opencage = 'cf3a00a9c9064546822a58a401e80d0f'  # Substitua pela sua chave de API do OpenCage

    endereco_completo = f'Escola {row["Escola"]}, {row["Endereço"]}, {row["Bairro"]}, {row["CEP"]}, Recife-PE'

    endpoint = 'https://api.opencagedata.com/geocode/v1/json'

    parametros = {
        'q': endereco_completo,
        'key': api_key_opencage
    }

    try:
        resposta = requests.get(endpoint, params=parametros)
        resposta.raise_for_status()  # Lança uma exceção se a resposta indicar um erro HTTP

        dados = resposta.json()

        if 'results' in dados and len(dados['results']) > 0:
            latitude = dados['results'][0]['geometry']['lat']
            longitude = dados['results'][0]['geometry']['lng']
            return latitude, longitude
        else:
            print(f"Erro: Nenhuma coordenada encontrada para o endereço: {endereco_completo}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None, None
    except Exception as e:
        print(f"Erro desconhecido: {e}")
        return None, None

# Criar duas novas colunas para latitude e longitude
df['Latitude'], df['Longitude'] = zip(*df.apply(obter_coordenadas_opencage, axis=1))

# Salvar DataFrame como arquivo CSV
df.to_csv('LatLongEscolasCageData.csv', index=False)
df

Erro: Nenhuma coordenada encontrada para o endereço: Escola PROFESSOR JOSUE DE CASTRO, MARTIN LUTHER KING 368 COQUE , ILHA JOANA BEZERRA, 50080090 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola REITOR JOAO ALFREDO, SENADOR JOSE HENRIQUE 160 , ILHA DO LEITE, 50070460 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola DE AGUA FRIA - ANEXO, OLIMPIO TORRES S/N , CAMPINA DO BARRETO, 52121050 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola JOSEFINA MARINHO, CORREGO JOSE GRANDE 1316 , ALTO JOSE DO PINHO, 52080040 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola ARQUITETO ALEXANDRE MUNIZ DE OLIVEIRA, ALTO DO VENANCIO 120 , CORREGO DO JENIPAPO, 52291023 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola BOLA NA REDE, DOUTOR ARTHUR ORLANDO DE ANDRADE BEZERRA 100 , GUABIRABA, 52490220 Recife-PE
Erro: Nenhuma coordenada encontrada para o endereço: Escola PROFESSOR ARIANO VILAR SUASSUNA, DON MANU

,idEscola,Escola,Endereço,Bairro,Telefone,RPA,CEP,Latitude,Longitude
0,1,ALMIRANTE SOARES DUTRA,CAMUTANGA 180,CABANGA,8133553880,1,50090380,-8.078802,-34.895299
1,2,ANA ROSA FALCAO DE CARVALHO,JOAO LYRA S/N,SANTO AMARO,8133556546,1,50050550,-27.688060,-48.778610
2,3,BOLSA ESCOLA MUNICIPAL,GENERAL ABREU E LIMA S/N,SANTO AMARO,NaN,1,50040210,-27.688060,-48.778610
3,4,CAP - BLOCO A DEGEP,FREI MATIAS TEVIS S/N,ILHA DO LEITE,NaN,1,50050470,19.000000,-70.666670
4,5,CAP - BLOCO A DEGRE,FREI MATIAS TEVIS S/N,ILHA DO LEITE,NaN,1,50050470,19.000000,-70.666670
...,...,...,...,...,...,...,...,...,...
422,423,UTEC MOVEL 6,ALMIRANTE NELSON FERNANDES S/N,BOA VIAGEM,8133550416,6,51030230,-8.053890,-34.881110
423,424,VILA DOS MILAGRES,SAO MIGUEL 14 MILAGRES,COHAB,8133554866,6,51345600,-8.053890,-34.881110
424,425,VILA OPERARIA DO RECIFE,EMILIO MONTEIRO FONSECA 1530,IBURA,8133556579,6,51240490,-7.356940,-35.440560
425,426,VILA SESAMO,VALE DO CARIRI 65 UR 05,COHAB,8133553160,6,51320180,-8.133686,-34.950133


In [15]:
# coordenadas = pd.read_csv('LatLongEscolasCageData.csv')

# coordenadas.isnull().sum()

parte = 19
total = 427

# Calculando a porcentagem
porcentagem = (parte / total) * 100

print(f"{100-porcentagem:.2f}% das coordenadas preenchidas usando CageData")

95.55% das coordenadas preenchidas usando CageData


In [19]:
df.drop_duplicates().isnull().sum()

idEscola      0
Escola        0
Endereço      0
Bairro        0
Telefone     24
RPA           0
CEP           6
Latitude     19
Longitude    19
dtype: int64